In [1]:
import pandas as pd
import glob

import sys

sys.path.append('..')
from git_repo_crawler import data_handler

In [8]:
import pandas as pd
import glob
from datetime import timedelta
import sys

sys.path.append('..')
from git_repo_crawler import data_handler

In [9]:
datapath = "../data/raw"
df = data_handler.load_all(datapath)

In [10]:
datecols = [c for c in df.columns if "date" in c.lower()]
for c in datecols:
    df[c] = pd.to_datetime(df[c])
    
df.dtypes

hash                               object
msg                                object
author_date           datetime64[ns, UTC]
author_timezone                     int64
committer_date        datetime64[ns, UTC]
committer_timezone                  int64
branches                           object
in_main_branch                       bool
merge                                bool
parents                            object
project_name                       object
project_path                      float64
data_source                        object
author_name                        object
author_email                       object
committer_name                     object
committer_email                    object
reference                          object
fpath                              object
dtype: object

In [11]:
# drop rows for hits in db and dev/db /*
df = pd.DataFrame(df[~df['fpath'].str.split('/').apply(lambda x: "db" in x)])

In [12]:
import re
# figure out what kind of reference each row is
df['reftype'] = df['reference'].apply(lambda x: re.findall(r"[\w']+", x)[0]).str.upper()

In [13]:
# clean up timezones
for role in ['author','committer']:
    k1 = f"{role}_date"
    k2 = f"{role}_timezone"
    
    df[k1] = df[k1] + df[k2].apply(lambda x: timedelta(seconds=x))
    df = df.drop(columns=[k2,])

# now that our times are normalized, find the earliest for each row
df['min_date'] = df[['author_date','committer_date']].min(axis=1)

# reorder columns
cols = ['reference', 'min_date','reftype', 'fpath', 'project_name', 'project_path', 
        'data_source', 'hash', 'msg', 'branches', 'in_main_branch', 'merge', 'parents',
        'author_name', 'author_email', 'author_date',
        'committer_name', 'committer_email',  'committer_date', 
       ]
df = df[cols]

# get rid of stuff we don't need
drop_cols = ['project_path','in_main_branch','merge','branches','parents', ]
df = df.drop(columns=drop_cols)

# we want to consolidate all the files changed in a commit for a vul
agg = {k: "first" for k in df.columns}
agg['fpath'] = list
df = df.groupby(['reference','hash']).aggregate(agg).reset_index(drop=True)

# sort ascending by date
df = df.sort_values(by='min_date',ascending=True)
# then drop duplicates for the same vul 
df = df.drop_duplicates(subset=["reference","project_name"],keep='first')
df['fpath'] = df['fpath'].apply(set).apply(list)

df

,reference,min_date,reftype,fpath,project_name,data_source,hash,msg,author_name,author_email,author_date,committer_name,committer_email,committer_date
7995,CVE-2013-1775,1970-01-01 00:02:02+00:00,CVE,[modules/post/osx/manage/sudo_password_bypass.rb],metasploit-framework,https://github.com/rapid7/metasploit-framework...,27c56cd3a3de14ffbb2fb1a9f30cd1bd1aab969b,Updates module to use native payload exe or a ...,Joe Vennix,Joe_Vennix@rapid7.com,1970-01-01 00:02:02+00:00,Joe Vennix,Joe_Vennix@rapid7.com,1970-01-01 00:02:02+00:00
15437,OSVDB-1,2005-06-05 00:33:38+00:00,OSVDB,[lib/msf/core/module/reference.rb.ut.rb],metasploit-framework,https://github.com/rapid7/metasploit-framework...,67cfc3d3861da1d996d14f32b666c1b5f77e8501,unit testage\n\n\ngit-svn-id: file:///home/svn...,Matt Miller,mmiller@hick.org,2005-06-05 00:33:38+00:00,Matt Miller,mmiller@hick.org,2005-06-05 00:33:38+00:00
16267,OSVDB-2100,2005-06-05 04:37:48+00:00,OSVDB,[modules/exploits/windows/msrpc_dcom_ms03_026.rb],metasploit-framework,https://github.com/rapid7/metasploit-framework...,04a9cbd7ad9bd16709daf9ff062f7215dc6f8888,moved\n\n\ngit-svn-id: file:///home/svn/incomi...,Matt Miller,mmiller@hick.org,2005-06-05 04:37:48+00:00,Matt Miller,mmiller@hick.org,2005-06-05 04:37:48+00:00
3364,CVE-2003-0344,2005-09-22 03:24:32+00:00,CVE,[modules/exploits/windows/ms03_020_ie_objectty...,metasploit-framework,https://github.com/rapid7/metasploit-framework...,fe0a151d7e65146deeb273460d0f4cc786055956,"ported ie object type, more work on http stuff...",Matt Miller,mmiller@hick.org,2005-09-22 03:24:32+00:00,Matt Miller,mmiller@hick.org,2005-09-22 03:24:32+00:00
16615,OSVDB-2967,2005-09-22 03:24:32+00:00,OSVDB,[modules/exploits/windows/ms03_020_ie_objectty...,metasploit-framework,https://github.com/rapid7/metasploit-framework...,fe0a151d7e65146deeb273460d0f4cc786055956,"ported ie object type, more work on http stuff...",Matt Miller,mmiller@hick.org,2005-09-22 03:24:32+00:00,Matt Miller,mmiller@hick.org,2005-09-22 03:24:32+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15265,CVE-2021-24146,2021-07-03 05:01:54+00:00,CVE,[exploits/php/webapps/50084.py],exploitdb,https://github.com/offensive-security/exploitd...,5bd61e68a2aec6442f1e9bdd364ec9e126b6980b,DB: 2021-07-03\n\n7 changes to exploits/shellc...,Offensive Security,info@exploit-db.com,2021-07-03 05:01:54+00:00,Offensive Security,info@exploit-db.com,2021-07-03 05:01:54+00:00
15051,CVE-2020-7750,2021-07-03 05:01:54+00:00,CVE,[exploits/multiple/webapps/50079.txt],exploitdb,https://github.com/offensive-security/exploitd...,5bd61e68a2aec6442f1e9bdd364ec9e126b6980b,DB: 2021-07-03\n\n7 changes to exploits/shellc...,Offensive Security,info@exploit-db.com,2021-07-03 05:01:54+00:00,Offensive Security,info@exploit-db.com,2021-07-03 05:01:54+00:00
15426,CVE-2021-35956,2021-07-03 05:01:54+00:00,CVE,[exploits/hardware/webapps/50080.txt],exploitdb,https://github.com/offensive-security/exploitd...,5bd61e68a2aec6442f1e9bdd364ec9e126b6980b,DB: 2021-07-03\n\n7 changes to exploits/shellc...,Offensive Security,info@exploit-db.com,2021-07-03 05:01:54+00:00,Offensive Security,info@exploit-db.com,2021-07-03 05:01:54+00:00
15266,CVE-2021-24155,2021-07-06 05:02:03+00:00,CVE,[exploits/php/webapps/50093.py],exploitdb,https://github.com/offensive-security/exploitd...,540825f14096091fb915bcf0bd3f007db249f1ce,DB: 2021-07-06\n\n11 changes to exploits/shell...,Offensive Security,info@exploit-db.com,2021-07-06 05:02:03+00:00,Offensive Security,info@exploit-db.com,2021-07-06 05:02:03+00:00


In [14]:
df.to_csv("../output/all.csv")

cve = df[df['reftype'] == "CVE"]
cve.to_csv("../output/cve.csv")

In [15]:
cve

,reference,min_date,reftype,fpath,project_name,data_source,hash,msg,author_name,author_email,author_date,committer_name,committer_email,committer_date
7995,CVE-2013-1775,1970-01-01 00:02:02+00:00,CVE,[modules/post/osx/manage/sudo_password_bypass.rb],metasploit-framework,https://github.com/rapid7/metasploit-framework...,27c56cd3a3de14ffbb2fb1a9f30cd1bd1aab969b,Updates module to use native payload exe or a ...,Joe Vennix,Joe_Vennix@rapid7.com,1970-01-01 00:02:02+00:00,Joe Vennix,Joe_Vennix@rapid7.com,1970-01-01 00:02:02+00:00
3364,CVE-2003-0344,2005-09-22 03:24:32+00:00,CVE,[modules/exploits/windows/ms03_020_ie_objectty...,metasploit-framework,https://github.com/rapid7/metasploit-framework...,fe0a151d7e65146deeb273460d0f4cc786055956,"ported ie object type, more work on http stuff...",Matt Miller,mmiller@hick.org,2005-09-22 03:24:32+00:00,Matt Miller,mmiller@hick.org,2005-09-22 03:24:32+00:00
3396,CVE-2003-0727,2005-11-25 04:11:22+00:00,CVE,[modules/exploits/windows/ftp/oracle9i_xdb_ftp...,metasploit-framework,https://github.com/rapid7/metasploit-framework...,cd8b9849ceba9a4e0d561606063791d9ab938dbb,Many fixes/updates/new ftp modules\nFixed the ...,HD Moore,hd_moore@rapid7.com,2005-11-25 04:11:22+00:00,HD Moore,hd_moore@rapid7.com,2005-11-25 04:11:22+00:00
3454,CVE-2004-0330,2005-11-25 04:36:56+00:00,CVE,[modules/exploits/windows/ftp/servu_mdtm.rb],metasploit-framework,https://github.com/rapid7/metasploit-framework...,7679d8e891023dbe162404d8887aa0bcaba45681,Dun dun dun.. da dun!\n\n\ngit-svn-id: file://...,HD Moore,hd_moore@rapid7.com,2005-11-25 04:36:56+00:00,HD Moore,hd_moore@rapid7.com,2005-11-25 04:36:56+00:00
3779,CVE-2005-2087,2005-11-26 00:00:09+00:00,CVE,[dev/porting/queue/ie_javaprxy.rb],metasploit-framework,https://github.com/rapid7/metasploit-framework...,c22ba98ad6277447c54aff0bd5d9816a41a97846,Exploit porting queue\n\n\ngit-svn-id: file://...,HD Moore,hd_moore@rapid7.com,2005-11-26 00:00:09+00:00,HD Moore,hd_moore@rapid7.com,2005-11-26 00:00:09+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15265,CVE-2021-24146,2021-07-03 05:01:54+00:00,CVE,[exploits/php/webapps/50084.py],exploitdb,https://github.com/offensive-security/exploitd...,5bd61e68a2aec6442f1e9bdd364ec9e126b6980b,DB: 2021-07-03\n\n7 changes to exploits/shellc...,Offensive Security,info@exploit-db.com,2021-07-03 05:01:54+00:00,Offensive Security,info@exploit-db.com,2021-07-03 05:01:54+00:00
15051,CVE-2020-7750,2021-07-03 05:01:54+00:00,CVE,[exploits/multiple/webapps/50079.txt],exploitdb,https://github.com/offensive-security/exploitd...,5bd61e68a2aec6442f1e9bdd364ec9e126b6980b,DB: 2021-07-03\n\n7 changes to exploits/shellc...,Offensive Security,info@exploit-db.com,2021-07-03 05:01:54+00:00,Offensive Security,info@exploit-db.com,2021-07-03 05:01:54+00:00
15426,CVE-2021-35956,2021-07-03 05:01:54+00:00,CVE,[exploits/hardware/webapps/50080.txt],exploitdb,https://github.com/offensive-security/exploitd...,5bd61e68a2aec6442f1e9bdd364ec9e126b6980b,DB: 2021-07-03\n\n7 changes to exploits/shellc...,Offensive Security,info@exploit-db.com,2021-07-03 05:01:54+00:00,Offensive Security,info@exploit-db.com,2021-07-03 05:01:54+00:00
15266,CVE-2021-24155,2021-07-06 05:02:03+00:00,CVE,[exploits/php/webapps/50093.py],exploitdb,https://github.com/offensive-security/exploitd...,540825f14096091fb915bcf0bd3f007db249f1ce,DB: 2021-07-06\n\n11 changes to exploits/shell...,Offensive Security,info@exploit-db.com,2021-07-06 05:02:03+00:00,Offensive Security,info@exploit-db.com,2021-07-06 05:02:03+00:00


In [26]:
def test(x):
    print("---")
    (ref,hash) = x
    print(f"ref={ref} hash={hash}")

df = df.sort_values(by="author_date")
df = df.drop_duplicates(subset="reference",keep="first")
df[["reference","hash"]].apply(test,axis=1)

---
ref=CVE-2013-1775 hash=27c56cd3a3de14ffbb2fb1a9f30cd1bd1aab969b
---
ref=OSVDB-1 hash=67cfc3d3861da1d996d14f32b666c1b5f77e8501
---
ref=OSVDB-2100 hash=04a9cbd7ad9bd16709daf9ff062f7215dc6f8888
---
ref=CVE-2003-0344 hash=fe0a151d7e65146deeb273460d0f4cc786055956
---
ref=OSVDB-2967 hash=fe0a151d7e65146deeb273460d0f4cc786055956
---
ref=OSVDB-8398 hash=501d7da9f91c9f8bc7d384a051ea5edbc1bf6fb2
---
ref=OSVDB-5248 hash=a5cde852994ac1c78aacb6eda16ac307a095d6ab
---
ref=OSVDB-875 hash=05a84cc7da02194dfa3cd86a21cd043ec3a127bc
---
ref=OSVDB-5762 hash=4edf57d73191557c6e0073b94f6b3278b2a69a24
---
ref=BID-12155 hash=973e7d16e8d653f5bda1e6c2131ba551b4d4e90b
---
ref=OSVDB-12811 hash=973e7d16e8d653f5bda1e6c2131ba551b4d4e90b
---
ref=OSVDB-12810 hash=973e7d16e8d653f5bda1e6c2131ba551b4d4e90b
---
ref=OSVDB-16049 hash=32d83b8cf4b8daf154852d14848a026a9520331e
---
ref=BID-13396 hash=7ae0c3555b331ad72459386d057b3585fb832788
---
ref=BID-8375 hash=cd8b9849ceba9a4e0d561606063791d9ab938dbb
---
ref=OSVDB-2449 hash=

7995     None
15437    None
16267    None
3364     None
16615    None
         ... 
15051    None
15264    None
15426    None
15266    None
13681    None
Length: 8563, dtype: object